1. histogram by cnn, plot by cnn
2. plot all busses and find idling areas / times
3. Query to identify single trip

## summary table
- Summary table by CNN 
- Summary table by stops (every second within 50 feet of a stop point, it will be a stop)
- periodic calculations, how often will they be peformed
- cron job to run in postgres dd?
- python function to start


## SQL queries
- volume per CNN
- route(trip) identification
- sopping and update stoping table
- point inside/near CNN polygon
- distance from polygon
-

## dropdown UI
- (done) timestamp with date fields
- how to update parameters with each field


## Implementation
1. upsert? function to tie each individual point to cnn, use postgres windowing fucntion. gropupby and min on distance to cnn (to associate cnn to points)
2. stops, companies and vehicles added to the db schema
3. summary of stops and frequency of stops, add table of stops
4. self report stops by changing a field, initially look at self reporting and see how accurate it is
- query to look at min/max time stamp with every group of points to classify stops
- flter for specific stop



### When should we persist data?
- summaries by CNN (every half hour at a 15min lag)
- 

- We should compare apples to apples both solutions

In [1]:
import sfmta
import pandas as pd
import os

os.environ['SHUTTLE_DB_USER'] = "postgres"
os.environ['SHUTTLE_DB_PASSWORD'] =''

In [2]:
conn = sfmta.db_connect()
#postgres is the username

In [ ]:
plate = 'CP96872'
start = '2017-09-19 11:50:45'
end = '2017-09-19 16:55:41'

sfmta.get_points_for_shuttle(conn, plate, start, end)

In [ ]:
query5 = '''
SELECT *
FROM  shuttle_locations
WHERE local_timestamp BETWEEN '2017-09-19 11:50:45'::timestamp
                 AND '2017-09-19 16:55:41'::timestamp;
'''
cursor = conn.cursor()
cursor.execute(query5)

rows = cursor.fetchall()

cols = ['shuttle_id', 'tech_id', 'comp_id', 'time', 'loc', 'cnn']
df = pd.DataFrame(rows, columns=cols)

print(len(df))
df.head()

In [ ]:
query2 = '''
SELECT *
FROM shuttle_locations
limit 10
'''
cursor = conn.cursor()
cursor.execute(query2)

rows = cursor.fetchall()
df = pd.DataFrame(rows)
df

In [ ]:
license_plate = 'CP96872'

query = '''SELECT local_timestamp, location[0], location[1] 
           FROM shuttle_locations l LEFT OUTER JOIN shuttles s 
           ON l.shuttle_id=s.id WHERE s.vehicle_license_plate = (%s)
          ORDER BY local_timestamp'''

cursor = conn.cursor()
cursor.execute(query2)

rows = cursor.fetchall()
df = pd.DataFrame(rows)
df

---

In [33]:
import sfmta
import json
import pandas as pd
from ipywidgets import Dropdown, HBox, VBox, Button, DatePicker, ColorPicker, IntSlider
from ipyleaflet import Map, Polyline, GeoJSON, Circle
import datetime

def clear_map():
    if 'm' in globals():
        m.close()
    else:
        pass
    
def clear_lat_longs():
    if 'lat_longs' in globals():
        lat_longs = None
    else:
        pass
    
def clear_time_slider():
    if 'time_slider' in globals():
        time_slider = None
    else:
        pass

def draw_update_map(b):
    clear_map()
    clear_lat_longs()
    plate = plate_dropdown.value
    start_date_time = str(start_date.value) + ' ' + start_time_dropdown.value
    end_date_time = str(end_date.value) + ' ' + end_time_dropdown.value
    global locations
    locations = sfmta.get_points_for_shuttle(conn, 
                                             plate, 
                                             start_date_time,
                                             end_date_time)
    
    zip_locations = zip(locations['lng'], locations['lat'], locations.index)
    global lat_longs
    lat_longs = [(x[0], x[1]) for x in zip_locations]
    index_value = find_index(lat_longs, time_slider.value)
    global m
    m = draw_map(lat_longs[:index_value])
    display(m)
    
button = Button(description="Draw/Update Map")
button.on_click(draw_update_map)
time_slider = IntSlider(min=0, max=100, step=1, description='% of Data', value=100)

def find_index(lat_longs, value):
    length = len(lat_longs)
    index = int(length * value/100)
    return index

def draw_map(lat_longs):
    center = [37.79481, -122.41186]
    zoom = 12
    m = Map(center=center, zoom=zoom)
    m.layout.height = '650px'
    pl = Polyline(locations=lat_longs)
    pl.color = path_color.value
    pl.fill_color = path_color.value
    m.add_layer(pl)
    return m

plates = sfmta.get_all_shuttles(conn)['LICENSE_PLATE'].unique()
plate_dropdown = Dropdown(options=plates, description='Plate')

# Add restrictions
def show_restrictions(b):
    polygons = 'vehiclerestrictions_wgs.json'
    
    with open(polygons) as f:
        polygons_json = json.load(f)

    global geojson
    geojson = GeoJSON(data=polygons_json)
    m.add_layer(geojson)
    

# Remove restrictions
def remove_restrictions(b):
    if 'geojson' in globals():
        m.remove_layer(geojson)
    else:
        pass
    
def download_data(b):
    if 'lat_longs' in globals():
        df = pd.DataFrame(data=lat_longs)
        df.to_excel('output.xlsx')
    else:
        pass
    
button_restrictions = Button(description="Show Restrictions")
button_restrictions.on_click(show_restrictions)
    
button_remove_restrictions = Button(description="Hide Restrictions")
button_remove_restrictions.on_click(remove_restrictions)

path_color = ColorPicker(description='Color')

export_data = Button(description="Download Data")
export_data.on_click(download_data)


def top_n_stops(b):
    from collections import Counter
    count_lat_longs = Counter(lat_longs).most_common()
    for lat_long in count_lat_longs[0:10]:
        location = lat_long[0]
        circle = Circle(location=location, radius=300)
        m.add_layer(circle)
    return m

button_add_top_stops = Button(description="Display Bus Idling")
button_add_top_stops.on_click(top_n_stops)

# Remove map button needs it's own function with a single argument
def remove_map(b):
    clear_map()

button_clear_map = Button(description="Remove Map")
button_clear_map.on_click(remove_map)

start = datetime.date.today()
end = datetime.date.today()
start_date = DatePicker(description='Start Date',value=start)
end_date = DatePicker(description='End Date', value=end)

start_time_dropdown = sfmta.time_drop_down()
start_time_dropdown.description = 'Start Time'
end_time_dropdown = sfmta.time_drop_down()
end_time_dropdown.description = 'End Time'

companies = sfmta.get_shuttles_companies(conn)['NAME'].unique()
company = Dropdown(options=companies, value='WeDriveU', description='Company')

VBox([HBox([company, button, button_restrictions, button_remove_restrictions]),
      HBox([plate_dropdown, export_data, button_add_top_stops, button_clear_map]),
      HBox([path_color]),
      HBox([start_date, start_time_dropdown]),
      HBox([end_date, end_time_dropdown]),
      HBox([time_slider]),
     ])

VBox(children=(HBox(children=(Dropdown(description='Company', index=5, options=('Storer Transportation Service DBA Storer Coachways DBA Storer San Francisco', 'Lux Leasing LLC', 'Kaiser Permanente', 'Hallcon Corporation   (Loop Transportation, Inc.)', 'Pure Luxury Transportation', 'WeDriveU', 'Royal Coach Tours', 'Compass Transportation', 'TRANSMETRO', 'Corinthian International Parking Services Inc. ', 'Bishop Ranch Transportation', 'Black Tie Transportaion', 'San Francisco Minibus'), value='WeDriveU'), Button(description='Draw/Update Map', style=ButtonStyle()), Button(description='Show Restrictions', style=ButtonStyle()), Button(description='Hide Restrictions', style=ButtonStyle()))), HBox(children=(Dropdown(description='Plate', options=('CP96872', 'CP96879', '76734M1', '14053T1', '8T47570', '8Z97897', '8Z98015', '8Z98150', '60595U1', '48361F1', 'CP83137', 'PLBUS60', 'CP96880', 'CP80923', '96562J1', '8T69481', '8Z98048', 'CP96868', '63488B2', '63002E1', '23485S1', '02418T1', '63689A1', '93057W1', '23486S1', '8Z56334', '96263J1', '59956U1', '08236Y1', 'PLBUS73', '8T47569', 'PLBUS84', 'PLBUS57', '59955U1', '8T69480', 'CP96884', '93120W1', '23965S1', 'CP77577', '', '31591E2', '58595C2', '8V53967', '62684X1', 'PLBUS76', 'CP96875', '96608J1', '8Z56587', 'EP00723', '44023L1', '93125W1', '8S75248', '64322B2', '8S63363', '15998C1', 'CP80515', '31588E2', '415640Z', '56055G1', '56054G1', '56056G1', '8V14304', '64943B2', '1654401', '23503U1', '87177R1', '8W02553', '15877V1', '16502C1', '98273k1', '93121W1', '84763A2', '44315L1', '63690A1', '63486B2', '63487B2', '96573J1', '8Z98043', '23502U1', '34763H2', '88358R1', '71947Z1', '04167T1', '08238Y1', '17447U1', '13861L1', '8Z98026', '11784N1', '03670T1', 'EP00725', '46984N1', '62686X1', '79508Y1', '64319B2', '08235Y1', '63301E1', '15336M1', '58368C2', '98084K1', '15925V1', '14237D2', '79505Y1', '03668T1', '04164T1', '84762A2', '02416T1', '83965D1', '88357R1', '08355Y1', '07757H1', '98387K1', '71946Z1', 'PLBUS56', '61862J1', '04165T1', '63687A1', '96403J1', 'BUJ6137', '98275k1', '8S75246', '69570X1', '8S63365', '96122J1', '58367C2', 'BUE3848', '78459N1', '98122S1', '69220X1', '63489B2', '51761B1', '34985H2', '8Y70385', '64320B2', '85844A2', '88360R1', '62679X1', '64097B2', '85651G1', '53600P1', 'EP00710', '66225X1', '98272k1', '65270V1', '48042S1', '96618J1', '80696N1', '8Z90217', '63142E1', '34764H2', '80693N1', '8Z56585', '61740B2', '96405J1', '71921Z1', '64079B2', '85513G1', '96480J1', '79506Y1', 'EP00711', '81199Y1', '96219J1', '64942B2', '58307C2', '66703X1', '85628G1', '58369C2', '96574J1', '64383B2', '60497U1', '62678X1', '33771A2', '08719Y1', '66230X1', '30689E2', '65274V1', '09469D2', '85785G1', '65275V1', '87178R1', '35368K1', '96402J1', '92396W1', '92571W1', '66957V1', '98778E2', '66484V1', 'EP00724', '80694N1', '63688A1', '87181R1', '8Z98046', '34987H2', '96406J1', '71922Z1', '35021H2', '92363W1', '8Y70386', '16543E1', '92361W1', '48362F1', '16280V1', 'CP80825', '09803D2', '33195A2', '08354Y1', '8Z97907', 'EP00709', '33193A2', '14326P1', '8Y70294', '08352Y1', 'CP96874', '23501U1', '30692E2', 'EP00726', '03603T1', '52035B1', '48382F1', '92364W1', '85787G1', '62683X1', '03669T1', '87180R1', '59410T1', '09500D2', '85784G1', '08237Y1', '16503C1', '09502D2', '14302D2', '98083K1', '68932X1', '33194A2', '88355R1', 'EP00708', '62677X1', '23207U1', '63295E1', '02455T1', '13859L1', '69569X1', '31975T1', '15876V1', '63584E1', '78461N1', '69568X1', '15878V1', '16500C1', '07851H1', '66222V1', '62681X1', '23504U1', '59499C2', '8Z56577', '92362W1', '62675X1', '19021U1', '34986H2', '48043S1', '93126W1', '58366C2', '66221V1', '62685X1', '59411T1', '61715J1', '62687X1', '30691E2', '79507Y1', '63719E1', '83863A2', '59959U1', '63486A1', '34988H2', '59498C2', '63908B2', '85443G1', 'PLBUS77', 'CP80824', '15337M1', '66223X1', '92452W1', '61499B2', '8Z97821', '8Z97908', '48381F1', '13863L1', '76421Y1', '96265J1', '09715D2', '59608U1', '96302J1', '95105V1', '96616J1', 'EP00707', '76512Y1', '30688E2', '9

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

Map(center=[37.79481, -122.41186], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), Polyline(color='black', fill_color='black', locations=[(37.7467872, -122.3969664), (37.7467744, -122.3967104), (37.746752000000001, -122.396672), (37.7467264, -122.3966336), (37.746697599999997, -122.3965568), (37.746687999999999, -122.3965312), (37.746659200000003, -122.3965056), (37.746435200000001, -122.3966848), (37.7459712, -122.3970944), (37.745539200000003, -122.3974528), (37.745590399999998, -122.39764479999999), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745657600000001, -122.397696), (37.745660800000003, -122.397696), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976832), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976704), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745660800000003, -122.3976576), (37.745750399999999, -122.397312), (37.745884799999999, -122.3970944), (37.746198399999997, -122.3968256), (37.746534400000002, -122.39654400000001), (37.746633600000003, -122.3964672), (37.746758399999997, -122.39654400000001), (37.746848, -122.3968256), (37.7469088, -122.39710719999999), (37.747033600000002, -122.397184), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747052799999999, -122.3972096), (37.747056000000001, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.747059200000002, -122.3972096), (37.746851200000002, -122.3977216), (37.746934400000001, -122.39781120000001), (37.746991999999999, -122.3977984), (37.746995200000001, -122.3977984), (37.747046400000002, -122.39778560000001), (37.7470912, -122.3977984), (37.747142400000001, -122.3978752), (37.747132800000003, -122.39802880000001), (37.747110399999997, -122.3984128), (37.747075199999998, -122.3987968), (37.746953599999998, -122.3991808), (37.746771199999998, -122.3997184), (37.746611199999997, -122.400256), (37.746464000000003, -122.400704), (37.746416000000004, -122.4008192), (37.746352000000002, -122.4009344), (37.746249599999999, -122.4012544), (37.746111999999997, -122.4017024), (37.746028799999998, -122.4019456), (37.746025600000003, -122.4019584), (37.746019199999999, -122.40197120000001), (37.746019199999999, -122.40197120000001), (37.7459968, -122.40204799999999), (37.746067199999999, -122.40226560000001), (37.746188799999999, -122.4024448), (37.746211199999998, -122.40250880000001), (37.746307199999997, -122.4026624), (37.746169600000002, -122.40282879999999), (37.745945599999999, -122.4028672), (37.745555199999998, -122.4029184), (37.745088000000003, -122.4029824), (37.744643199999999, -122.40307199999999), (37.744246400000002, -122.4031616), (37.743971199999997, -122.40323840000001), (37.743910399999997, -122.4032512), (37.743625600000001, -122.4032256), (37.743222400000001, -122.40326399999999), (37.742848000000002, -122.403328), (37.742432000000001, -122.4034048), (37.742182399999997, -122.40345600000001), (37.742156799999997, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421376, -122.40345600000001), (37.7421

In [ ]:
complaint, shuttle bus on this street at this time (placard, time, location)

entry level person to select whever they know

tool for shuttle table
- make, model, company, day/time, location, 

shuttle picker

figure out directionality from points (play button on how to plot)

In [ ]:
tutorials
- here is how to make a new button/widget
- take user stories and create docs on how to answer that question

In [ ]:
for this day, for this operator, what are the potential violations
- be specific about the 
- have the ability to exclude

In [ ]:
count and amount of time on a street, filter by that
- i only want to see violations on a street that have more than x number of points


In [ ]:
group cnns for each vehicle per day, 
how many violations occured, how many had a count greater than 50

# Use cases

- track down complaint
- shuttle volumes, trips, stops
- admin penalities
- list of shuttles for a given area, then plot the routes -- get list of shuttles, then plot all routes for that list


In [ ]:
import pandas as pd

In [ ]:
import psycopg2
import os

In [ ]:
cursor = conn.cursor()
cursor.execute("""SELECT * from shuttle_locations s
                           join providers p
                           on p.id = s.shuttle_id
                           limit 20;""")
rows = cursor.fetchall()
pd.DataFrame(rows)

In [ ]:
column_headers = {
    'shuttle_locations': ['shuttle_id', 'tech_provider_id', 'shuttle_company_id',
                          'local_timestamp', 'location', 'cnn']
}

In [ ]:
pd.DataFrame(rows)

In [ ]:
m

In [ ]:
block_polygons = 'StreetPolygonsWGS.json'

with open(block_polygons) as f:
    block_json = json.load(f)


block_geojson = GeoJSON(data=block_json, hover_style={'fillColor': 'red'})
m.add_layer(block_geojson)
m

In [ ]:
block_json_tmp = block_json
block_json_tmp['features'] = block_json['features'][0:100] 

block_geojson_tmp = GeoJSON(data=block_json_tmp, hover_style={'fillColor': 'red'})
m.add_layer(block_geojson_tmp)

In [ ]:
block_json_tmp

In [ ]:
target_cnn = 224201

In [ ]:
for feature in block_json_tmp['features']:
    print(feature['properties']['CNN'])

In [ ]:

    
# Add blocks
def show_blocks(b):
    blocks = 'tmp2.json'
    
    with open(blocks) as f:
        block_json = json.load(f)

    global block_geojson
    block_geojson = GeoJSON(data=block_json)
    m.add_layer(block_geojson)

button_blocks = Button(description="Show Blocks")
button_blocks.on_click(show_blocks)


In [ ]:
# blocks = 'tmp2.json'
    
# with open(blocks) as f:
#     block_json = json.load(f)

# global block_geojson
# block_geojson = GeoJSON(data=block_json)
# m.add_layer(block_geojson)
# m

In [ ]:
block_polygons = 'StreetPolygonsWGS.json'
with open(blocks) as f:
    block_json = json.load(f)

global block_geojson
block_geojson = GeoJSON(data=block_json)
m.add_layer(block_geojson)
m